# Importing required libraries

In [ ]:
import sys
import pprint
import pandas as pd
import numpy as np
from scipy.special import softmax

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
import sklearn
import os

import shutil

from os import listdir
from os.path import isfile, join

import time

In [ ]:
# Setting path for importing required functions for data processing

sys.path.append("/home/jupyter/sonam/adhd_nlp/final_notebook_folder/data_processing")
sys.path.append("/home/jupyter/sonam/adhd_nlp/final_notebook_folder/data_processing")

In [ ]:
# Functions required for data processing

import final_process_text
import final_transform_textfiles

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
# using simpletransformer ai library

from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Import Data

In [ ]:
label_of_interest = "BT_yn"

In [ ]:
originalTextData = final_transform_textfiles.extractOriginalText("/home/jupyter/data/cohort_2to6/Text files/combined_text")

annotatedXMIs = final_transform_textfiles.extractXMIAnnotation("/home/jupyter/data/cohort_2to6/XMI files/combined")


In [ ]:
print(annotatedXMIs['anon_id'].value_counts(ascending=False))

# Data Processing 

In [ ]:
# Deleting last row from 'originalTextData' and 'annotatedXMIS' dataframe as the last row is just checpoint row mentioned above
# for  ANNON_ID 10040 and hence deleting it would result in final cohort size of 432 as needed for new results.

In [ ]:
# dropping last row in text and XMI pandas dataframe.
#originalTextData.drop(originalTextData.tail(-1).index,inplace=True)
# annotatedXMIs.drop(annotatedXMIs.tail(-1).index,inplace=True)

originalTextData = originalTextData[:-1]
annotatedXMIs = annotatedXMIs[:-1]


In [ ]:
# creating single label_of_interest "BT_yn" column using other columns

annotatedXMIs['BT_yn'] = np.where((annotatedXMIs['Counsel_Parent_BT'] == 1) | (annotatedXMIs['Counsel_Handout_BT'] == 1) | (annotatedXMIs['Refer_Parent_BT'] == 1) | (annotatedXMIs['Refer_School_BT'] == 1), 1, 0)
annotatedXMIs['BT_yn'].value_counts()

In [ ]:
# merging data from both files 
data = originalTextData.merge(annotatedXMIs, on = "file", how = "right")

In [ ]:
# using imported function sectionize() for processing notes text data

data['extractText'] = data['note_des'].apply(lambda x: final_process_text.sectionize(x)[1])

In [ ]:
# using imported function clean_text() for processing notes text data

data['extractText'] = data['extractText'].apply(lambda x: final_process_text.clean_text(x))

In [ ]:
data = data.loc[:, ['extractText',label_of_interest]]\
       .rename(columns = {'extractText':'text',
                          label_of_interest: 'label'})

In [ ]:
X = data.loc[:, 'text']
y = data.loc[:, 'label']

# Split the Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117, stratify = y)
X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 117, stratify = y_train)

In [ ]:
val_train = pd.concat([X_val_train, y_val_train], axis = 1)
val_test = pd.concat([X_val_test, y_val_test], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)

# test.head()

# Loading saved model and evaluating

In [ ]:
# loading saved weights from the training  for defining the model
model = ClassificationModel("bert", "./final_biobert_output_dir_new_cohort",use_cuda = True, num_labels = 2)

#Evaluate on Validation set (89 samples)

In [ ]:

result, model_outputs, wrong_predictions = model.eval_model(val_test, f1 = f1_score, 
                                                            recall = sklearn.metrics.recall_score,
                                                            precision = sklearn.metrics.precision_score,
                                                            auc = sklearn.metrics.roc_auc_score,
                                                            accuracy = sklearn.metrics.accuracy_score)
result

In [ ]:
predictions, probabilities = model.predict(val_test['text'].tolist())
val_test['predictions'] = predictions
val_test['probabilities'] = [x[1] for x in np.array([softmax(element) for element in probabilities])]

#Function to create the precion-recall-f1 score 

In [ ]:
def precision_recall_metrics(true_label , pred_prob):
#     true_label= val_test['label']
#     pred_prob = val_test['probabilities']

    precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_label, pred_prob)
    precision = precision[:-1]
    recall = recall[:-1]
    f1 = 2*(precision*recall)/(precision+recall)
    results_DF = pd.DataFrame(data = {'precision': precision, 
                                       'recall': recall,
                                       'f1' : f1,
                                      'thresholds':thresholds})
    print(results_DF)


#Function to create a confusion matrix 

In [ ]:
def confusion_matrix_thr(threshold_final, true_label, pred_prob):
    
    pred_label = (pred_prob >= threshold_final)

    pred_label = pred_label.values.astype(int)

    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(true_label, pred_label, normalize='true').ravel()
    print(sklearn.metrics.classification_report(true_label, pred_label))

    print("tn:",tn)
    print("tp:",tp)
    print("fn:",fn)
    print("fp:",fp)
    
    return pred_label

In [ ]:
print("The dataframe with all the metrics and threshold to set a threshold value which gives maximum precision with decent recall and f1 score.")

precision_recall_metrics(val_test['label'], val_test['probabilities'])

#selected row 38 with threshold of 0.001842 for further calculations

#Creating confusion matrix for Validation set(89 samples)

In [ ]:
true_label_val= val_test['label']
pred_prob_val = val_test['probabilities']

In [ ]:

confusion_matrix_thr(0.001842,true_label_val,pred_prob_val)

#Creating confusion matrix for Test set(127 samples)

In [ ]:
test_predictions, test_probabilities = model.predict(test['text'].tolist())

test['predictions'] = test_predictions
test['probabilities'] = [x[1] for x in np.array([softmax(element) for element in test_probabilities])]

In [ ]:
test_true_label= test['label']
test_pred_prob = test['probabilities']

In [ ]:
# Saving predictions obtained for test set using threshold value in arr
arr =confusion_matrix_thr(0.001842,test_true_label,test_pred_prob)

In [ ]:
# Updating predictions for test set with new predictions obtained according to threshold value

test['predictions']= arr

#Function to get the dataframe for missclassified samples 

In [ ]:
def get_WP(test):
    
    index= test.index
    condition = ((test['label'] != test['predictions']))
    missclassified_indices = index[condition]

    missclassified_indices_list = missclassified_indices.tolist() 
    miss_df = test.loc[test.index.isin(missclassified_indices_list)]
    return miss_df

In [ ]:
test_miss_df = get_WP(test)

wrong_predictions = test_miss_df.to_records(index=True)
len(wrong_predictions)

#saving the misclassified notes in file for test set

In [ ]:
with open('/home/jupyter/sonam/final_result_files/final_threshold_test_misclassification_new_cohort.txt', mode='wt', encoding='utf-8') as myfile:
    list=["index","tokenLength","text", "truth", "error"]
    myfile.writelines(str(list));
    myfile.writelines("\n");
    for x in range(len(wrong_predictions)): 
        list=[];
        for y in range(3): 
            if(y==0):
                list.append(wrong_predictions[x][y])
            elif (y ==2):
                truth=wrong_predictions[x][y];
                list.append(truth);
                if truth==1: list.append("fn")
                elif truth==0: list.append("fp")
            elif(y==1):
                list.append(len(model.tokenizer(wrong_predictions[x][y])['input_ids']));
                list.append(wrong_predictions[x][y]);
        myfile.writelines(str(list));
        myfile.writelines("\n");
myfile.close

#Checking saved model on Train Set(207 samples)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val_train, f1 = f1_score,  
                                                            recall = sklearn.metrics.recall_score,
                                                            precision = sklearn.metrics.precision_score,
                                                            auc = sklearn.metrics.roc_auc_score,
                                                           accuracy = sklearn.metrics.accuracy_score)
result

#Getting precision, recall, and thresholds for Test SEt (127 samples )

In [ ]:
precision, recall, thresholds = sklearn.metrics.precision_recall_curve(test['label'], test['predictions'])
precision = precision[:-1]
recall = recall[:-1]
thresholdDF = pd.DataFrame(data = {'precision': precision, 'recall': recall, 'thresholds':thresholds})
thresholdDF['f1_score'] = 2*(thresholdDF['precision']*thresholdDF['recall'])/(thresholdDF['precision'] + thresholdDF['recall'])

In [ ]:
max(thresholdDF['f1_score'])